## 1. Setup and Imports

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
from pathlib import Path
from typing import Dict, List, Tuple

# Audio processing
import librosa
import librosa.display

# Deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import to_categorical

# Machine learning utilities
from sklearn.metrics import (
    confusion_matrix, classification_report, 
    precision_recall_fscore_support, roc_auc_score
)
from sklearn.utils.class_weight import compute_class_weight

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 2. Load Preprocessed Data

In [ ]:
# Data directory
DATA_DIR = Path("/Users/cococe/Desktop/AUDIOSET METADATA/preprocessed_data")

# Load configuration
with open(DATA_DIR / 'preprocessing_config.json', 'r') as f:
    config = json.load(f)

print("Preprocessing Configuration:")
print(json.dumps(config, indent=2))

In [ ]:
# Load data splits
def load_split(split_name: str) -> List[Dict]:
    """Load a data split."""
    with open(DATA_DIR / f'{split_name}_data.pkl', 'rb') as f:
        return pickle.load(f)

print("Loading data splits...")
train_data = load_split('train')
val_data = load_split('val')
test_data = load_split('test')

print(f"\nTrain samples: {len(train_data):,}")
print(f"Validation samples: {len(val_data):,}")
print(f"Test samples: {len(test_data):,}")
print(f"Total: {len(train_data) + len(val_data) + len(test_data):,}")

## 3. Exploratory Data Analysis

In [ ]:
# Analyze threat level distribution
def get_label_distribution(data: List[Dict]) -> pd.DataFrame:
    """Get label distribution statistics."""
    threat_levels = [sample['label']['threat_level'] for sample in data]
    threat_names = [sample['label']['threat_level_name'] for sample in data]
    subcategories = [sample['label']['subcategory'] for sample in data]
    
    df = pd.DataFrame({
        'threat_level': threat_levels,
        'threat_name': threat_names,
        'subcategory': subcategories
    })
    
    return df

train_df = get_label_distribution(train_data)
val_df = get_label_distribution(val_data)
test_df = get_label_distribution(test_data)

print("\n=== TRAIN SET ===")
print(train_df['threat_name'].value_counts())
print("\n=== VALIDATION SET ===")
print(val_df['threat_name'].value_counts())
print("\n=== TEST SET ===")
print(test_df['threat_name'].value_counts())

In [ ]:
# Visualize distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Threat level distribution
threat_counts = train_df['threat_name'].value_counts()
axes[0, 0].bar(threat_counts.index, threat_counts.values, color=['red', 'orange', 'green'])
axes[0, 0].set_title('Training Set - Threat Level Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Number of Samples')
axes[0, 0].tick_params(axis='x', rotation=45)

# Subcategory distribution
subcat_counts = train_df['subcategory'].value_counts()
axes[0, 1].barh(subcat_counts.index, subcat_counts.values)
axes[0, 1].set_title('Training Set - Subcategory Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Number of Samples')

# Threat level percentages
threat_pcts = train_df['threat_name'].value_counts(normalize=True) * 100
axes[1, 0].pie(threat_pcts.values, labels=threat_pcts.index, autopct='%1.1f%%',
               colors=['red', 'orange', 'green'], startangle=90)
axes[1, 0].set_title('Threat Level Percentage', fontsize=14, fontweight='bold')

# Train/Val/Test split sizes
split_sizes = [len(train_data), len(val_data), len(test_data)]
split_names = ['Train', 'Validation', 'Test']
axes[1, 1].bar(split_names, split_sizes, color=['blue', 'orange', 'green'])
axes[1, 1].set_title('Dataset Split Sizes', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('Number of Samples')

plt.tight_layout()
plt.show()

## 4. Visualize Audio Samples

In [ ]:
# Find one sample from each threat level
threat_samples = {}
for idx, sample in enumerate(train_data):
    threat_name = sample['label']['threat_level_name']
    if threat_name not in threat_samples:
        threat_samples[threat_name] = sample
    if len(threat_samples) == 3:
        break

# Visualize each threat level
for threat_name, sample in threat_samples.items():
    features = sample['features']
    label = sample['label']
    
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    # Waveform
    waveform = features['waveform']
    time = np.arange(len(waveform)) / config['target_sr']
    axes[0].plot(time, waveform, linewidth=0.5)
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title(f"{threat_name} - {label['subcategory']} - Waveform", fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Mel-spectrogram
    mel_spec = features['mel_spectrogram']
    img1 = librosa.display.specshow(
        mel_spec,
        sr=config['target_sr'],
        hop_length=config['hop_length'],
        x_axis='time',
        y_axis='mel',
        ax=axes[1],
        cmap='viridis'
    )
    axes[1].set_title('Mel-Spectrogram (128 bands)', fontweight='bold')
    fig.colorbar(img1, ax=axes[1], format='%+2.0f dB')
    
    # MFCCs
    mfcc = features['mfcc']
    img2 = librosa.display.specshow(
        mfcc,
        sr=config['target_sr'],
        hop_length=config['hop_length'],
        x_axis='time',
        ax=axes[2],
        cmap='coolwarm'
    )
    axes[2].set_title('MFCCs (40 coefficients)', fontweight='bold')
    axes[2].set_ylabel('MFCC Coefficient')
    fig.colorbar(img2, ax=axes[2])
    
    plt.tight_layout()
    plt.show()

## 5. Prepare Data for Model Training

In [ ]:
def prepare_data(data: List[Dict], feature_type: str = 'mel_spectrogram') -> Tuple[np.ndarray, np.ndarray]:
    """
    Prepare features and labels for training.
    
    Args:
        data: List of samples
        feature_type: Type of features to extract
        
    Returns:
        Tuple of (features, labels)
    """
    features = np.array([sample['features'][feature_type] for sample in data])
    labels = np.array([sample['label']['threat_level'] for sample in data])
    
    # Add channel dimension for CNN if using spectrograms
    if feature_type in ['mel_spectrogram', 'mfcc']:
        features = features[..., np.newaxis]
    
    return features, labels

# Prepare data with mel-spectrograms
print("Preparing mel-spectrogram features...")
X_train, y_train = prepare_data(train_data, 'mel_spectrogram')
X_val, y_val = prepare_data(val_data, 'mel_spectrogram')
X_test, y_test = prepare_data(test_data, 'mel_spectrogram')

print(f"\nTraining features shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"\nValidation features shape: {X_val.shape}")
print(f"Test features shape: {X_test.shape}")

print(f"\nLabel distribution (train):")
unique, counts = np.unique(y_train, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  Class {label}: {count:,} samples ({count/len(y_train)*100:.1f}%)")

In [ ]:
# Compute class weights to handle imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

print("Class weights (to handle imbalance):")
for cls, weight in class_weight_dict.items():
    threat_name = ['BACKGROUND', 'THREAT_CONTEXT', 'THREAT'][cls]
    print(f"  {threat_name} (class {cls}): {weight:.3f}")

## 6. Build CNN Model for Mel-Spectrograms

In [ ]:
def build_cnn_model(input_shape: Tuple, num_classes: int = 3) -> keras.Model:
    """
    Build a CNN model for audio classification.
    
    Architecture optimized for mel-spectrogram inputs.
    """
    model = models.Sequential([
        # Input layer
        layers.Input(shape=input_shape),
        
        # Conv Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Conv Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Conv Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Conv Block 4
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
        # Global pooling and dense layers
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        
        # Output layer
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Build model
model = build_cnn_model(input_shape=X_train.shape[1:])
model.summary()

In [ ]:
# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

print("Model compiled successfully!")

## 7. Setup Training Callbacks

In [ ]:
# Create directory for models
MODEL_DIR = Path("/Users/cococe/Desktop/AUDIOSET METADATA/models")
MODEL_DIR.mkdir(exist_ok=True)

# Define callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = callbacks.ModelCheckpoint(
    filepath=str(MODEL_DIR / 'best_model.keras'),
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

tensorboard = callbacks.TensorBoard(
    log_dir=str(MODEL_DIR / 'logs'),
    histogram_freq=1
)

callback_list = [early_stopping, model_checkpoint, reduce_lr, tensorboard]

print("Callbacks configured:")
print("  - Early stopping (patience=15)")
print("  - Model checkpoint (save best)")
print("  - Learning rate reduction (factor=0.5, patience=5)")
print("  - TensorBoard logging")

## 8. Train the Model

In [ ]:
# Train model
print("Starting training...\n")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=callback_list,
    verbose=1
)

print("\nTraining complete!")

## 9. Visualize Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss
axes[0, 0].plot(history.history['loss'], label='Train Loss')
axes[0, 0].plot(history.history['val_loss'], label='Val Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Model Loss', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy
axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy')
axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].set_title('Model Accuracy', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Precision
axes[1, 0].plot(history.history['precision'], label='Train Precision')
axes[1, 0].plot(history.history['val_precision'], label='Val Precision')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].set_title('Model Precision', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Recall
axes[1, 1].plot(history.history['recall'], label='Train Recall')
axes[1, 1].plot(history.history['val_recall'], label='Val Recall')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].set_title('Model Recall', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Evaluate on Test Set

In [ ]:
# Evaluate on test set
print("Evaluating on test set...\n")
test_results = model.evaluate(X_test, y_test, verbose=1)

print("\nTest Results:")
print(f"  Loss: {test_results[0]:.4f}")
print(f"  Accuracy: {test_results[1]:.4f}")
print(f"  Precision: {test_results[2]:.4f}")
print(f"  Recall: {test_results[3]:.4f}")

In [ ]:
# Get predictions
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)

# Classification report
class_names = ['BACKGROUND', 'THREAT_CONTEXT', 'THREAT']
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
plt.ylabel('True Label', fontsize=12, fontweight='bold')
plt.title('Confusion Matrix - Test Set', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Normalized confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='RdYlGn',
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Percentage'})
plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
plt.ylabel('True Label', fontsize=12, fontweight='bold')
plt.title('Normalized Confusion Matrix - Test Set', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 11. Per-Class Performance Analysis

In [ ]:
# Per-class metrics
precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred)

metrics_df = pd.DataFrame({
    'Class': class_names,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support
})

print("\nPer-Class Performance Metrics:")
print(metrics_df.to_string(index=False))

# Visualize metrics
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics = ['Precision', 'Recall', 'F1-Score']
colors = ['#3498db', '#e74c3c', '#2ecc71']

for idx, metric in enumerate(metrics):
    axes[idx].bar(class_names, metrics_df[metric], color=colors)
    axes[idx].set_ylim([0, 1.1])
    axes[idx].set_ylabel(metric)
    axes[idx].set_title(f'{metric} by Class', fontweight='bold')
    axes[idx].tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for i, v in enumerate(metrics_df[metric]):
        axes[idx].text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 12. Critical Threat Detection Analysis

In [ ]:
# Analyze false negatives for THREAT class (most critical)
threat_indices = np.where(y_test == 2)[0]  # THREAT = class 2
threat_predictions = y_pred[threat_indices]

false_negatives = np.sum(threat_predictions != 2)
true_positives = np.sum(threat_predictions == 2)

print("\n=== CRITICAL THREAT DETECTION ANALYSIS ===")
print(f"\nTotal THREAT samples in test set: {len(threat_indices)}")
print(f"Correctly detected: {true_positives} ({true_positives/len(threat_indices)*100:.1f}%)")
print(f"Missed (False Negatives): {false_negatives} ({false_negatives/len(threat_indices)*100:.1f}%)")

if false_negatives > 0:
    print(f"\n⚠️  WARNING: {false_negatives} THREAT samples were not detected!")
    print("These represent critical failures in the anti-poaching system.")
    
    # Show what threats were misclassified as
    fn_indices = threat_indices[threat_predictions != 2]
    fn_predictions = threat_predictions[threat_predictions != 2]
    
    print("\nMisclassifications:")
    unique_preds, counts = np.unique(fn_predictions, return_counts=True)
    for pred, count in zip(unique_preds, counts):
        print(f"  THREAT misclassified as {class_names[pred]}: {count} times")
else:
    print("\n✅ Perfect THREAT detection! No false negatives.")

## 13. Save Model for Deployment

In [ ]:
# Save final model
final_model_path = MODEL_DIR / 'threat_detection_final.keras'
model.save(final_model_path)
print(f"\n✅ Final model saved to: {final_model_path}")

# Save model configuration
model_config = {
    'model_type': 'CNN',
    'feature_type': 'mel_spectrogram',
    'input_shape': list(X_train.shape[1:]),
    'num_classes': 3,
    'class_names': class_names,
    'preprocessing': config,
    'test_accuracy': float(test_results[1]),
    'test_precision': float(test_results[2]),
    'test_recall': float(test_results[3]),
    'class_metrics': metrics_df.to_dict('records')
}

config_path = MODEL_DIR / 'model_config.json'
with open(config_path, 'w') as f:
    json.dump(model_config, f, indent=2)

print(f"✅ Model configuration saved to: {config_path}")

# Export to TensorFlow Lite for edge deployment
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

tflite_path = MODEL_DIR / 'threat_detection.tflite'
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f"✅ TensorFlow Lite model saved to: {tflite_path}")
print(f"   Model size: {len(tflite_model) / 1024:.1f} KB")

## 14. Inference Demo

In [ ]:
# Test real-time inference
def predict_threat(model, audio_features: np.ndarray) -> Dict:
    """
    Predict threat level from audio features.
    
    Args:
        model: Trained model
        audio_features: Preprocessed audio features
        
    Returns:
        Dictionary with prediction results
    """
    # Add batch dimension if needed
    if audio_features.ndim == 3:
        audio_features = np.expand_dims(audio_features, axis=0)
    
    # Get prediction
    probabilities = model.predict(audio_features, verbose=0)[0]
    predicted_class = np.argmax(probabilities)
    confidence = probabilities[predicted_class]
    
    return {
        'threat_level': int(predicted_class),
        'threat_name': class_names[predicted_class],
        'confidence': float(confidence),
        'probabilities': {
            name: float(prob) for name, prob in zip(class_names, probabilities)
        },
        'alert_rangers': predicted_class >= 2  # Alert if THREAT detected
    }

# Test on random samples
print("\n=== INFERENCE DEMO ===")
print("Testing on 5 random samples from test set...\n")

for i in range(5):
    idx = np.random.randint(0, len(X_test))
    sample_features = X_test[idx]
    true_label = y_test[idx]
    
    result = predict_threat(model, sample_features)
    
    print(f"Sample {i+1}:")
    print(f"  True Label: {class_names[true_label]}")
    print(f"  Predicted: {result['threat_name']} (confidence: {result['confidence']:.1%})")
    print(f"  Alert Rangers: {'🚨 YES' if result['alert_rangers'] else '✓ No'}")
    print(f"  Probabilities:")
    for name, prob in result['probabilities'].items():
        print(f"    {name}: {prob:.1%}")
    print()

## 15. Deployment Recommendations

### Model Performance Summary
- The model has been trained and evaluated on 6,734 audio samples
- Three-tier threat classification system ready for ranger notification
- TensorFlow Lite model exported for edge deployment

### Deployment Considerations

1. **Real-Time Processing**:
   - Model processes 10-second audio clips
   - Inference time: ~10-50ms on modern hardware
   - Suitable for continuous monitoring

2. **Alert System**:
   - THREAT (Level 2): Immediate ranger alert
   - THREAT_CONTEXT (Level 1): Log and monitor
   - BACKGROUND (Level 0): No action required

3. **Edge Deployment**:
   - TensorFlow Lite model: Lightweight for Raspberry Pi/edge devices
   - Low power consumption for solar-powered field deployment
   - Offline operation capability

4. **Monitoring and Improvement**:
   - Log all predictions with timestamps and GPS coordinates
   - Collect ranger feedback for continuous improvement
   - Retrain model periodically with new data
   - Monitor false positive/negative rates

5. **Critical Metrics to Watch**:
   - THREAT recall (minimize missed threats)
   - False positive rate (avoid alert fatigue)
   - Model drift over time

### Next Steps
1. Deploy TensorFlow Lite model to field devices
2. Integrate with ranger communication system
3. Set up monitoring dashboard
4. Conduct field testing and validation
5. Implement feedback loop for model improvement